![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# Start coding here...
db_bank = pd.read_csv('bank_marketing.csv')
db_bank.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [2]:
db_client=db_bank[['client_id','age','job','marital','education','credit_default','housing','loan']]
db_campaign=db_bank[['client_id','campaign','duration','pdays','previous','poutcome','y','month','day']]
db_economics=db_bank[['client_id','emp_var_rate','cons_price_idx','euribor3m','nr_employed']]


In [3]:
db_client.head()

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [4]:
db_campaign.tail(150)

,client_id,campaign,duration,pdays,previous,poutcome,y,month,day
41038,41038,2,226,999,1,failure,yes,oct,29
41039,41039,5,297,999,0,nonexistent,yes,oct,22
41040,41040,1,215,6,1,success,yes,oct,27
41041,41041,1,129,999,0,nonexistent,no,oct,27
41042,41042,2,211,6,3,success,yes,oct,7
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,nonexistent,yes,nov,30
41184,41184,1,383,999,0,nonexistent,no,nov,6
41185,41185,2,189,999,0,nonexistent,no,nov,24
41186,41186,1,442,999,0,nonexistent,yes,nov,17


In [5]:
db_economics.head()

,client_id,emp_var_rate,cons_price_idx,euribor3m,nr_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


## Cleaning the data
**Renaming columns and check for it**

In [6]:

db_client.rename(columns={"client_id":"id"},inplace=True)
db_client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [7]:
db_campaign.rename(columns={"duration":"contact_duration","previous":"previous_campaign_contacts","y":"campaign_outcome","poutcome":"previous_outcome","campaign":"number_contacts"},inplace=True)
db_campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,0,1,261,999,0,nonexistent,no,may,13
1,1,1,149,999,0,nonexistent,no,may,19
2,2,1,226,999,0,nonexistent,no,may,23
3,3,1,151,999,0,nonexistent,no,may,27
4,4,1,307,999,0,nonexistent,no,may,3


In [8]:
db_economics.rename(columns={"euribor3m":"euribor_three_months","nr_employed":"number_employed"},inplace=True)
db_economics.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


**Replace "." with "_" at column education from client table**

In [9]:

db_client["education"]=db_client["education"].str.replace(".","_")


In [10]:
db_client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin.,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


**Replace "unkown" with NaN at column education**

In [11]:
db_client["education"]=db_client["education"].replace("unknown",np.NaN)


In [12]:
db_client.tail(20)

,id,age,job,marital,education,credit_default,housing,loan
41168,41168,38,entrepreneur,married,university_degree,no,no,no
41169,41169,62,services,married,high_school,no,yes,no
41170,41170,40,management,divorced,university_degree,no,yes,no
41171,41171,33,student,married,professional_course,no,yes,no
41172,41172,31,admin.,single,university_degree,no,yes,no
41173,41173,62,retired,married,university_degree,no,yes,no
41174,41174,62,retired,married,university_degree,no,yes,no
41175,41175,34,student,single,NaN,no,yes,no
41176,41176,38,housemaid,divorced,high_school,no,yes,yes
41177,41177,57,retired,married,professional_course,no,yes,no


**convert (success/failure) and (yes/no) in previous_outcome and campaign_outcome into 0/1 and to NAN in case of 'nonexistant' at previous_outcom**

In [13]:
#we will start with "campaign_outcome"
# P.S if you run this cell again it will return null
db_campaign["campaign_outcome"]=db_campaign["campaign_outcome"].map({"no":0,"yes":1})


In [14]:
db_campaign.tail(150)

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
41038,41038,2,226,999,1,failure,1,oct,29
41039,41039,5,297,999,0,nonexistent,1,oct,22
41040,41040,1,215,6,1,success,1,oct,27
41041,41041,1,129,999,0,nonexistent,0,oct,27
41042,41042,2,211,6,3,success,1,oct,7
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,nonexistent,1,nov,30
41184,41184,1,383,999,0,nonexistent,0,nov,6
41185,41185,2,189,999,0,nonexistent,0,nov,24
41186,41186,1,442,999,0,nonexistent,1,nov,17


In [15]:
#Now we will go on with "previous_outcome"
db_campaign["previous_outcome"]=db_campaign["previous_outcome"].map({"failure":0,"success":1})
db_campaign["previous_outcome"]=db_campaign["previous_outcome"].replace("nonexistant",np.NaN)

In [16]:
db_campaign.tail(150)

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
41038,41038,2,226,999,1,0.0,1,oct,29
41039,41039,5,297,999,0,NaN,1,oct,22
41040,41040,1,215,6,1,1.0,1,oct,27
41041,41041,1,129,999,0,NaN,0,oct,27
41042,41042,2,211,6,3,1.0,1,oct,7
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,NaN,1,nov,30
41184,41184,1,383,999,0,NaN,0,nov,6
41185,41185,2,189,999,0,NaN,0,nov,24
41186,41186,1,442,999,0,NaN,1,nov,17


**Now we are going to create a new column for datetime which have format of "year-month-day" and we will set year to 2022**

In [17]:
db_campaign["year"]=2022

In [18]:
db_campaign["last_contact_date"] = pd.to_datetime(db_campaign["year"].astype(str) + "-" + db_campaign["month"].str.capitalize() + "-" + db_campaign["day"].astype(str))

In [19]:
db_campaign.tail(150)

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day,year,last_contact_date
41038,41038,2,226,999,1,0.0,1,oct,29,2022,2022-10-29
41039,41039,5,297,999,0,NaN,1,oct,22,2022,2022-10-22
41040,41040,1,215,6,1,1.0,1,oct,27,2022,2022-10-27
41041,41041,1,129,999,0,NaN,0,oct,27,2022,2022-10-27
41042,41042,2,211,6,3,1.0,1,oct,7,2022,2022-10-07
...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,NaN,1,nov,30,2022,2022-11-30
41184,41184,1,383,999,0,NaN,0,nov,6,2022,2022-11-06
41185,41185,2,189,999,0,NaN,0,nov,24,2022,2022-11-24
41186,41186,1,442,999,0,NaN,1,nov,17,2022,2022-11-17


In [20]:
db_campaign["last_contact_date"] = pd.to_datetime(db_campaign["last_contact_date"]).dt.strftime("%Y-%b-%d")
db_campaign.tail(150)

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day,year,last_contact_date
41038,41038,2,226,999,1,0.0,1,oct,29,2022,2022-Oct-29
41039,41039,5,297,999,0,NaN,1,oct,22,2022,2022-Oct-22
41040,41040,1,215,6,1,1.0,1,oct,27,2022,2022-Oct-27
41041,41041,1,129,999,0,NaN,0,oct,27,2022,2022-Oct-27
41042,41042,2,211,6,3,1.0,1,oct,7,2022,2022-Oct-07
...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,NaN,1,nov,30,2022,2022-Nov-30
41184,41184,1,383,999,0,NaN,0,nov,6,2022,2022-Nov-06
41185,41185,2,189,999,0,NaN,0,nov,24,2022,2022-Nov-24
41186,41186,1,442,999,0,NaN,1,nov,17,2022,2022-Nov-17


**Now we no use of columns day, month and year we will delete these columns**

In [21]:
db_campaign.drop(["month","day","year"],axis=1,inplace=True)

In [22]:
db_campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,999,0,NaN,0,2022-May-13
1,1,1,149,999,0,NaN,0,2022-May-19
2,2,1,226,999,0,NaN,0,2022-May-23
3,3,1,151,999,0,NaN,0,2022-May-27
4,4,1,307,999,0,NaN,0,2022-May-03


**Now we will save the Data to a csv file**

In [23]:
db_client.to_csv("client.csv",index=False)
db_campaign.to_csv("campaign.csv",index=False)
db_economics.to_csv("economics.csv",index=False)